In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
classifier_id = 30

In [3]:
stack = 'MD591'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [ ]:
wait_num_nodes(16)

In [7]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': classifier_id},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
               cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 302.817464 seconds


In [33]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

downscale = 8

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v3.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) #

Resampling scoremaps ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 247.409529 seconds


In [ ]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
viz_downscale = 8

run_distributed(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d -d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v3.py'),
                'stack': stack,
                 'setting': classifier_id,
                'add_label_text': '-a' if add_label_text else '',
                'downscale': viz_downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [46]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 36s

constructing score volumes ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 36.286749 seconds


In [48]:
t = time.time()
sys.stderr.write('compute score volume gradients...')

run_distributed4(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct', 'compute_score_volume_gradients_v4.py'),
                'stack': stack,
                'setting': classifier_setting,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70 seconds

compute score volume gradients...

NameError: name 'classifier_setting' is not defined

In [47]:
! tail /home/ubuntu/stderr_2.log

File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
Load scoremaps: 0.51 seconds
Create score volume: 1.35 seconds
Child returned 0
1.22 seconds.
Child returned 0
0.38 seconds.
